In [1]:
%load_ext autoreload
%autoreload 2

# Experiments with EASE model

In [2]:
from pathlib import Path

import scipy.sparse
import numpy as np

import src.io as io
import src.evaluation as evaluation
import src.evaluation.splits
import src.evaluation.base

from src.algorithm.baseline.ease import EASE

## Datasets

In [3]:
## Uncomment corresponding directory.
## RETARGET True for Frappe, False for the others

# DATA_DIR, RETARGET = Path('../../../data/CARS/Mobile_Frappe/'), True
DATA_DIR, RETARGET = Path('../../../data/CARS/Food_com/'), False
# DATA_DIR, RETARGET = Path('../../../data/CARS/TripAdvisor/'), False

In [4]:
# Shouldn't need to change this info
INTERACTIONS = DATA_DIR / 'interactions.csv'

SEED = 123456
SEED2 = 78910

ITEM_ID = 'item'
USER_ID = 'user'

## Define algorithm and hyperparameter ranges

In [5]:
ALG = EASE

L2 = [10**p for p in range(1, 5)]
# L2 = [0.1, 0.5, 1, 5, 10, 15, 20, 50]
# L2 = [50, 80, 100, 150, 300, 500]

HYPERPARAMS = {'l2': L2}
HYPERPARAMS

{'l2': [10, 100, 1000, 10000]}

## Parse data

In [6]:
data = io.parse_interactions_with_context(INTERACTIONS, item_id=ITEM_ID, user_id=USER_ID)
data.df

,userId,itemId,season,weekday
0,19211,6280,1,1
1,17684,6280,2,2
2,12110,6280,3,3
3,7184,6280,3,1
4,17837,6280,2,4
...,...,...,...,...
388357,895,7799,3,7
388358,4100,7799,4,2
388359,2940,7799,1,4
388360,757,7799,2,2


## Make train/val split for hyperparam tuning

In [7]:
Xtrain, Xval_in, Xval_out = evaluation.splits.leave_one_out_split_non_context(data, seed=SEED)

## Perform gridsearch on validation set

In [8]:
%%time
score, best_hyperparams = evaluation.base.gridsearch(ALG, Xtrain, Xval_in, Xval_out, HYPERPARAMS, retarget=RETARGET)
f"Best score of {score} achieved with {best_hyperparams}."

  0%|          | 0/4 [00:00<?, ?it/s]

Training model EASE with hyperparameters {'l2': 10}
Evaluating with 22178 users
MRR@5 0.014
MRR@20 0.016
Average Recall@5 0.021
Average Recall@20 0.041
Training model EASE with hyperparameters {'l2': 100}
Evaluating with 22178 users
MRR@5 0.017
MRR@20 0.019
Average Recall@5 0.026
Average Recall@20 0.051
Training model EASE with hyperparameters {'l2': 1000}
Evaluating with 22178 users
MRR@5 0.017
MRR@20 0.02
Average Recall@5 0.027
Average Recall@20 0.057
Training model EASE with hyperparameters {'l2': 10000}
Evaluating with 22178 users
MRR@5 0.016
MRR@20 0.02
Average Recall@5 0.027
Average Recall@20 0.061
CPU times: user 1h 37min 45s, sys: 6min 1s, total: 1h 43min 47s
Wall time: 8min 25s


"Best score of 0.019928540274315322 achieved with {'l2': 1000}."

## Evaluate model with optimal hyperparams with KFolds Cross validation

In [9]:
%%time
alg = ALG(**best_hyperparams)
results = evaluation.base.kFoldsEval(alg, data, nr_folds=5, seed=SEED2, retarget=RETARGET)

Evaluating with 22178 users
Evaluating with 22178 users
Evaluating with 22178 users
Evaluating with 22178 users
Evaluating with 22178 users
MRR@5 0.016 (0.0)
MRR@20 0.019 (0.0)
Average Recall@5 0.026 (0.001)
Average Recall@20 0.058 (0.001)
CPU times: user 2h 3min 40s, sys: 6min 57s, total: 2h 10min 37s
Wall time: 10min 29s
